# Retail Demo Store - Search Workshop

Welcome to the Retail Demo Store Search Workshop. In this module we'll be configuring the Retail Demo Store Search service to allow searching for product data via Elasticsearch. An Elasticsearch domain should already be provisioned for you in your AWS environment.

Recommended Time: 20 Minutes

## Setup

To get started, we need to perform a bit of setup. Walk through each of the following steps to configure your environment to interact with the Amazon Personalize Service.

### Import Dependencies and Setup Boto3 Python Clients

Througout this workshop we will need access to some common libraries and clients for connecting to AWS services.

In [1]:
# Import Dependencies

import boto3
import json
import pandas as pd
import time
import requests
import csv

from random import randint
from botocore.exceptions import ClientError

# Setup Clients

servicediscovery = boto3.client('servicediscovery')
ssm = boto3.client('ssm')
es_service = boto3.client('es')

## Create Index and Bulk Index Product Data

### Get Products Service Instance

We will be creating a new Elasticsearch Index and indexing our product data so that our users can search for products. To do this, first we will be pulling our Product data from [Products Service](https://github.com/aws-samples/retail-demo-store/tree/master/src/products) that is deployed as part of the Retail Demo Store. To connect to the Products Service we will use Service Discovery to discover an instance of the Products Service, and then connect directly to that service instances to access our data.

In [2]:
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='products',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

products_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Service Instance IP: {}'.format(products_service_instance))

Service Instance IP: 10.215.10.226


#### Download and Explore the Products Dataset

Now that we have the IP address of one of our Products Service instances, we can connect to it and download our product catalog. To more easily explore our data, we will convert the json response form the Products Service into a dataframe and print it as a table. 

In [3]:
response = requests.get('http://{}/products/all'.format(products_service_instance))
products = response.json()
products_df = pd.DataFrame(products)
pd.set_option('display.max_rows', 5)

products_df

,id,url,sk,name,category,style,description,price,image,gender_affinity,current_stock,featured
0,e1669081-8ffc-4dec-97a6-e9176d7f6651,http://d3idkbp2p2okv.cloudfront.net/#/product/...,,Sans Pareil Scarf,apparel,scarf,Sans pareil scarf for women,124.99,http://d3idkbp2p2okv.cloudfront.net/images/app...,F,12,NaN
1,cfafd627-7d6b-43a5-be05-4c7937be417d,http://d3idkbp2p2okv.cloudfront.net/#/product/...,,Chef Knife,housewares,kitchen,A must-have for your kitchen,57.99,http://d3idkbp2p2okv.cloudfront.net/images/hou...,NaN,9,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2447,12f93a36-e282-4445-92ae-356eb6a560fd,http://d3idkbp2p2okv.cloudfront.net/#/product/...,,Roses Arrangement,floral,arrangement,Roses arrangement grown sustainably on our org...,98.99,http://d3idkbp2p2okv.cloudfront.net/images/flo...,NaN,19,NaN
2448,23e77743-ff28-400a-811f-e08d3cab5e45,http://d3idkbp2p2okv.cloudfront.net/#/product/...,,Ultrachic Scarf,apparel,scarf,Ultrachic scarf for women for every occasion,123.99,http://d3idkbp2p2okv.cloudfront.net/images/app...,F,7,NaN


### Install Elasticsearch Python Library

We will use the Python Elasticsearch library to connect to our Amazon Elasticsearch cluster, create a new index, and then bulk index our product data. First, we need to install the Elasticsearch library into our environment.

In [4]:
!pip install --upgrade pip
!pip install elasticsearch

  Using cached pip-21.0.1-py3-none-any.whl (1.5 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.3.3
    Uninstalling pip-20.3.3:
      Successfully uninstalled pip-20.3.3
     |████████████████████████████████| 325 kB 25.4 MB/s eta 0:00:01


### Find Elasticsearch Domain Endpoint

Before we can configure the Elasticsearch client, we need to determine the endpoint for the Elasticsearch domain created in your AWS environment. We will accomplish this by looking for the Elasticsearch domain with tag key of `Name` and tag value of `retaildemostore`. This tag was associated with the Amazon Elasticsearch domain that was created when the project was deployed to your AWS account.

In [5]:
elasticsearch_domain_endpoint = None

domains_response = es_service.list_domain_names()

for domain_name in domains_response['DomainNames']:
    describe_response = es_service.describe_elasticsearch_domain(
        DomainName=domain_name['DomainName']
    )
    
    tags_response = es_service.list_tags(ARN=describe_response['DomainStatus']['ARN'])

    domain_match = False
    for tag in tags_response['TagList']:
        if tag['Key'] == 'Name' and tag['Value'] == 'retaildemostore':
            domain_match = True
            break
            
    if domain_match:
        elasticsearch_domain_endpoint = describe_response['DomainStatus']['Endpoints']['vpc']
        break;

print('Elasticsearch domain endpoint: ' + str(elasticsearch_domain_endpoint))

if not elasticsearch_domain_endpoint:
    raise Exception('Elasticsearch domain endpoint could not be determined. Ensure Elasticsearch domain has been successfully created and has "retaildemostore" tag before continuing.')

Elasticsearch domain endpoint: vpc-retaild-elasti-5calm9svzdp5-egl24hvepbj2ckau6hfdgzm3mq.us-east-1.es.amazonaws.com


### Configure and Create Elasticsearch Client

In [6]:
from elasticsearch import Elasticsearch

ES_HOST = {
    'host' : elasticsearch_domain_endpoint,
    'port' : 443,
    'scheme' : 'https',
}
INDEX_NAME = 'products'
TYPE_NAME = 'product'
ID_FIELD = 'id'

es = Elasticsearch(hosts = [ES_HOST])

### Prepare Product Data for Indexing

Batch products into chunks to avoid timeouts.

In [7]:
bulk_datas = [] 
bulk_data = []

bulk_datas.append(bulk_data)

max_data_len = 100

for product in products:
    data_dict = product

    op_dict = {
        "index": {
            "_index": INDEX_NAME, 
            "_type": TYPE_NAME, 
            "_id": data_dict[ID_FIELD]
        }
    }
    bulk_data.append(op_dict)
    bulk_data.append(data_dict)
    
    if len(bulk_data) >= max_data_len:
        bulk_data = []
        bulk_datas.append(bulk_data)

### Check for and Delete Existing Indexes

If the products index already exists, we'll delete it so everything gets rebuilt from scratch.

In [8]:
if es.indices.exists(INDEX_NAME):
    print("Deleting '%s' index..." % (INDEX_NAME))
    res = es.indices.delete(index = INDEX_NAME)
    print(" response: '%s'" % (res))
else:
    print('Index does not exist. Nothing to do.')

Index does not exist. Nothing to do.


### Create Index

In [9]:
request_body = {
    "settings" : {
        "number_of_shards": 1,
        "number_of_replicas": 0
    }
}
print("Creating '%s' index..." % (INDEX_NAME))
res = es.indices.create(index = INDEX_NAME, body = request_body)
print(" response: '%s'" % (res))

Creating 'products' index...
 response: '{'acknowledged': True, 'shards_acknowledged': True, 'index': 'products'}'


### Perform Bulk Indexing

In [10]:
print("Bulk indexing...")
for bulk_data in bulk_datas:
    res = es.bulk(index = INDEX_NAME, body = bulk_data, refresh = True)
    
print("Done")

Bulk indexing...
Done


### Validate Results Through Elasticsearch

To verify that the products have been successfully indexed, let's perform a wildcard search for `beard*` directly against the Elasticsearch index.

In [11]:
res = es.search(index = INDEX_NAME, body={"query": {"wildcard": { "name": "brush*"}}})
print(json.dumps(res, indent=2))

{
  "took": 768,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 7,
      "relation": "eq"
    },
    "max_score": 1.0,
    "hits": [
      {
        "_index": "products",
        "_type": "product",
        "_id": "d16df944-ea75-43c3-b54b-6738731c081b",
        "_score": 1.0,
        "_source": {
          "id": "d16df944-ea75-43c3-b54b-6738731c081b",
          "url": "http://d3idkbp2p2okv.cloudfront.net/#/product/d16df944-ea75-43c3-b54b-6738731c081b",
          "sk": "",
          "name": "Lip Brush",
          "category": "beauty",
          "style": "grooming",
          "description": "Delicately calibrated hairs for consistency of application",
          "price": 32,
          "image": "http://d3idkbp2p2okv.cloudfront.net/images/beauty/d16df944-ea75-43c3-b54b-6738731c081b.jpg",
          "gender_affinity": "F",
          "current_stock": 2
        }
      },
      {
        

## Validate Results Through Search Service

Finally, let's verify that the Retail Demo Store's [Search service](https://github.com/aws-samples/retail-demo-store/tree/master/src/search) can successfully query from the Elasticsearch index as well.

### Discover Search Service

First we need to get the address to the [Search service](https://github.com/aws-samples/retail-demo-store/tree/master/src/search).

In [12]:
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='search',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

search_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Service Instance IP: {}'.format(products_service_instance))

Service Instance IP: 10.215.10.226


### Call Search Service

Let's call the service's index page which simply echos the service name.

In [13]:
!curl {search_service_instance}

Search Service

Finally, let's do the same `beard` search through the Search service. We should get back the same item IDs as the direct Elasticsearch query above.

In [14]:
!curl {search_service_instance}/search/products?searchTerm='brush'

[{"itemId": "39945ad0-57c9-4c28-a69c-532d5d167202"}, {"itemId": "4296626c-fbb0-42b4-9a50-b6c6c16095f3"}, {"itemId": "6b229d13-22a7-44d3-a13c-fd98dda49217"}, {"itemId": "f91ec34f-a08e-4408-8bb0-592bdd09375c"}, {"itemId": "8bc9e7d5-123c-46e4-a4f5-abeb79a77b3f"}, {"itemId": "d16df944-ea75-43c3-b54b-6738731c081b"}, {"itemId": "89728417-5269-403d-baa3-04b59cdffd0a"}, {"itemId": "b2f9ea63-f7d4-4944-84ce-208164326c8e"}, {"itemId": "1513d9f0-bb81-4b0c-bab5-20e8ea67c104"}]

## Workshop Complete

**Congratulations!** You have completed the first Retail Demo Store workshop where we indexed the products from the Retail Demo Store's Products microservice in an Elasticsearch domain index. This domain is used by the Retail Demo Store's Search microservice to process search queries from the Web user interface. To see this in action, open the Retail Demo Store's web UI in a new browser tab/window and enter a value in the search field at the top of the page.

### Next Step

Move on to the **[1-Personalization](../1-Personalization/1.1-Personalize.ipynb)** workshop where we will learn how to train machine learning models using Amazon Personalize to produce personalized product recommendations to users and add the ability to provide personalized reranking of products.